# Using Markdown with OpenTURNS

In this example, we use the Markdown format to pretty-print
results from an uncertainty quantification study.
We first show how the Markdown function from the IPython module
works.
Then we show how this can be used within a uncertainty quantification
study.

## References

- https://github.com/openturns/openturns/pull/2415

In [15]:
from IPython.display import Markdown, HTML
import openturns as ot
from openturns.usecases import stressed_beam

In [16]:
markdown = ""
# Itemized list
markdown += "- item\n"
markdown += "- item\n"
markdown += "\n"
# Table
markdown += "| Index | Value |\n"
markdown += "|-------|-------|\n"
n = 5
for i in range(n):
    markdown += "| %5s | %5s |\n" % (i, 1 + i)
markdown += "\n"
Markdown(markdown)

- item
- item

| Index | Value |
|-------|-------|
|     0 |     1 |
|     1 |     2 |
|     2 |     3 |
|     3 |     4 |
|     4 |     5 |



## Format a reliability study

In this section, we format the results of a reliability study.
Before doing this, we must see that the library already provides
pretty-print features which can be accessed through various
objects.
The most simple pretty-print feature is given by the
`Sample`.


In [17]:
data = ot.Sample(5, 3)
data

,v0,v1,v2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0



This can be convenient to format any result that can be
stored within a `Sample`, i.e. any data set that
has the shape of an array of floats.
There are, however, situations where this framework is not
general enough.
For example, we may want to print strings, integers,
lists of strings or list of integers (e.g. :class:`~openturns.Indices`)
and these data types cannot be stored in a `Sample`.
In the next example, we want to print an interval: this cannot
be done easily using the `Sample` class.
We could, of course, print the lower and upper bounds
into two different columns of the array, but this is less
straightforward to understand as an interval.

The next function performs a reliability study based on the
stressed beam.
Given the number of outer loops, the maximum coefficient of variation
and the block size, the function returns the result of the probability
simulation algorithm.
The function uses a Monte-Carlo experiment.



In [18]:
def computeReliabilityResult(maximumOuterSamplingSize, maximumCoV, blockSize=1):
    sm = stressed_beam.AxialStressedBeam()
    limitStateFunction = sm.model
    inputRandomVector = ot.RandomVector(sm.distribution)
    outputRandomVector = ot.CompositeRandomVector(limitStateFunction, inputRandomVector)
    myEvent = ot.ThresholdEvent(outputRandomVector, ot.Less(), 0.0)
    experiment = ot.MonteCarloExperiment()
    algoMC = ot.ProbabilitySimulationAlgorithm(myEvent, experiment)
    algoMC.setMaximumOuterSampling(maximumOuterSamplingSize)
    algoMC.setBlockSize(blockSize)
    algoMC.setMaximumCoefficientOfVariation(maximumCoV)
    algoMC.run()
    psaResult = algoMC.getResult()
    return psaResult

Use the function.


In [19]:
maximumOuterSamplingSize = 100
maximumCoV = 0.0
psaResult = computeReliabilityResult(maximumOuterSamplingSize, maximumCoV)
psaResult

probabilityEstimate=3.000000e-02 varianceEstimate=2.910000e-04 standard deviation=1.71e-02 coefficient of variation=5.69e-01 confidenceLength(0.95)=6.69e-02 outerSampling=100 blockSize=1

We increase the sample size and compute the
probability estimate and the corresponding confidence interval.
The output of this algorithm is a `markdown` string which contains
the results inside a Markdown table.

In [20]:
sampleSizeList = []
numberOfRepetitions = 10
sampleSizeFactor = 2
alpha = 0.05
maximumOuterSamplingSize = 2
markdown = ""
markdown += "| n | Pf | %.0f%%  C.I. |\n" % (100 * (1.0 - alpha))
markdown += "|---|----|--------------|\n"
for i in range(numberOfRepetitions):
    maximumOuterSamplingSize *= sampleSizeFactor
    psaResult = computeReliabilityResult(maximumOuterSamplingSize, maximumCoV)
    probability = psaResult.getProbabilityEstimate()
    pflen = psaResult.getConfidenceLength(1 - alpha)
    pfLower = probability - pflen / 2
    pfUpper = probability + pflen / 2
    markdown += "| %5d | %.3e | [%10.3e, %10.3e] |\n" % (
        maximumOuterSamplingSize,
        probability,
        pfLower,
        pfUpper,
    )
print(markdown)

| n | Pf | 95%  C.I. |
|---|----|--------------|
|     4 | 0.000e+00 | [ 0.000e+00,  0.000e+00] |
|     8 | 0.000e+00 | [ 0.000e+00,  0.000e+00] |
|    16 | 0.000e+00 | [ 0.000e+00,  0.000e+00] |
|    32 | 9.375e-02 | [-7.241e-03,  1.947e-01] |
|    64 | 4.687e-02 | [-4.910e-03,  9.866e-02] |
|   128 | 7.812e-03 | [-7.440e-03,  2.306e-02] |
|   256 | 3.125e-02 | [ 9.936e-03,  5.256e-02] |
|   512 | 2.734e-02 | [ 1.322e-02,  4.147e-02] |
|  1024 | 2.930e-02 | [ 1.897e-02,  3.963e-02] |
|  2048 | 2.832e-02 | [ 2.114e-02,  3.550e-02] |



In [21]:
# %%
# Pretty-print
Markdown(markdown)

| n | Pf | 95%  C.I. |
|---|----|--------------|
|     4 | 0.000e+00 | [ 0.000e+00,  0.000e+00] |
|     8 | 0.000e+00 | [ 0.000e+00,  0.000e+00] |
|    16 | 0.000e+00 | [ 0.000e+00,  0.000e+00] |
|    32 | 9.375e-02 | [-7.241e-03,  1.947e-01] |
|    64 | 4.687e-02 | [-4.910e-03,  9.866e-02] |
|   128 | 7.812e-03 | [-7.440e-03,  2.306e-02] |
|   256 | 3.125e-02 | [ 9.936e-03,  5.256e-02] |
|   512 | 2.734e-02 | [ 1.322e-02,  4.147e-02] |
|  1024 | 2.930e-02 | [ 1.897e-02,  3.963e-02] |
|  2048 | 2.832e-02 | [ 2.114e-02,  3.550e-02] |


In [22]:
# %%
# HTML
sampleSizeList = []
numberOfRepetitions = 10
sampleSizeFactor = 2
alpha = 0.05
maximumOuterSamplingSize = 2
html = "<table>\n"
html += "  <tr>\n"
html += "    <th> n </th>\n"
html += "    <th> Pf </th>\n"
html += "    <th> %.0f%%  C.I. </th>\n" % (100 * (1.0 - alpha))
html += "  </tr>\n"
for i in range(numberOfRepetitions):
    maximumOuterSamplingSize *= sampleSizeFactor
    psaResult = computeReliabilityResult(maximumOuterSamplingSize, maximumCoV)
    probability = psaResult.getProbabilityEstimate()
    pflen = psaResult.getConfidenceLength(1 - alpha)
    pfLower = probability - pflen / 2
    pfUpper = probability + pflen / 2
    html += "  <tr>\n"
    html += f"    <td> {maximumOuterSamplingSize} </td>\n"
    html += f"    <td> {probability:.3e} </td>\n"
    html += f"    <td> [{pfLower:10.3e}, {pfUpper:10.3e}] </td>\n"
    html += "  <tr>\n"
html += "</table>\n"
print(html)

<table>
  <tr>
    <th> n </th>
    <th> Pf </th>
    <th> 95%  C.I. </th>
  </tr>
  <tr>
    <td> 4 </td>
    <td> 0.000e+00 </td>
    <td> [ 0.000e+00,  0.000e+00] </td>
  <tr>
  <tr>
    <td> 8 </td>
    <td> 0.000e+00 </td>
    <td> [ 0.000e+00,  0.000e+00] </td>
  <tr>
  <tr>
    <td> 16 </td>
    <td> 0.000e+00 </td>
    <td> [ 0.000e+00,  0.000e+00] </td>
  <tr>
  <tr>
    <td> 32 </td>
    <td> 3.125e-02 </td>
    <td> [-2.903e-02,  9.153e-02] </td>
  <tr>
  <tr>
    <td> 64 </td>
    <td> 3.125e-02 </td>
    <td> [-1.138e-02,  7.388e-02] </td>
  <tr>
  <tr>
    <td> 128 </td>
    <td> 3.125e-02 </td>
    <td> [ 1.108e-03,  6.139e-02] </td>
  <tr>
  <tr>
    <td> 256 </td>
    <td> 4.297e-02 </td>
    <td> [ 1.813e-02,  6.781e-02] </td>
  <tr>
  <tr>
    <td> 512 </td>
    <td> 9.766e-03 </td>
    <td> [ 1.248e-03,  1.828e-02] </td>
  <tr>
  <tr>
    <td> 1024 </td>
    <td> 1.562e-02 </td>
    <td> [ 8.029e-03,  2.322e-02] </td>
  <tr>
  <tr>
    <td> 2048 </td>
    <td> 2.588

In [23]:
# %%
# Pretty-print
HTML(html)

n,Pf,95% C.I.
4,0.000e+00,"[ 0.000e+00, 0.000e+00]"
8,0.000e+00,"[ 0.000e+00, 0.000e+00]"
16,0.000e+00,"[ 0.000e+00, 0.000e+00]"
32,3.125e-02,"[-2.903e-02, 9.153e-02]"
64,3.125e-02,"[-1.138e-02, 7.388e-02]"
128,3.125e-02,"[ 1.108e-03, 6.139e-02]"
256,4.297e-02,"[ 1.813e-02, 6.781e-02]"
512,9.766e-03,"[ 1.248e-03, 1.828e-02]"
1024,1.562e-02,"[ 8.029e-03, 2.322e-02]"
2048,2.588e-02,"[ 1.900e-02, 3.276e-02]"
